In [ ]:
#Putting them both together..

In [ ]:
#First, the ticker file reader + text adjuster:

In [ ]:
import requests
import pprint
from bs4 import BeautifulSoup
from datetime import datetime as dt
import pandas as pd
import csv
import os
import time

In [ ]:
os.chdir("C:/Users/Mac/Desktop/Investing/Financial Data/Ticker Lists/")
os.getcwd()

In [ ]:
#Reading file for list of tickers
ticker_list_tsxv = []

with open('TSXV Tickers.csv', 'r') as file:
    csv_file = csv.reader(file)
    for row in csv_file:
        ticker_list_tsxv.append(row)    

#Check a sample of the csv data
print(ticker_list_tsxv[1])

In [ ]:
#isolating relevant data
ticker_list_test = []

for x in range(len(ticker_list_tsxv)):
    ticker_list_test.append(ticker_list_tsxv[x][0])

ticker_list_test = ticker_list_test[1:]

In [ ]:
#Adding the ".V" for Venture tickers and adjusting other tickers to proper form
tsx_list = []

for x in ticker_list_test:
    if "." in x:
        tsx_list.append(x.replace(".","-") + ".TO")
    
    
    else:
        tsx_list.append(x + ".TO")

In [ ]:
#Next, the actual data scraper:

In [ ]:
#Defining formatting tools

class FormattingTool:
    
    def __init__(self):
        pass
    
    def value_to_float(self, x):
        if type(x) == float or type(x) == int:
            return x
        if 'K' in x:
            if len(x) > 1:
                return float(x.replace('K', '')) * 1000
            return 1000.0
        if 'M' in x:
            if len(x) > 1:
                return float(x.replace('M', '')) * 1000000
            return 1000000.0
        if 'B' in x:
            if len(x) > 1:
                return float(x.replace('B', '')) * 1000000000
        if 'T' in x:
            return float(x.replace('T', '')) * 1000000000000
        return 0.0
    
    def create_pd_dataframe(self):
        pd.options.display.float_format = '{:,}'.format
        df_columns = ["Ticker","Date","Price","Shares Outstanding","Market Cap","EV","EBITDA","EV/EBITDA","D/E","BV/Share","Total Cash","Total Liabilities","Total Revenue","Operating Cash Flow", "BS Recent Date", "CF Recent Date", "IS Recent Date"]
        stock_info_df = pd.DataFrame(data = None, columns = df_columns, index = [0])    
        return stock_info_df

In [ ]:
#Defining data scraping functions
#note: cash flows have been pulled for TTM, rather than simply most recent year-end


class FinancialDataScraper:
    def __init__(self, ticker, stock_data):
        self.ticker = ticker

        
    #define function to pull most recent dates for all financial statements of ticker company
    def financials_date_scrape(self):
        
        recent_dates_temp = []
        #balance sheet date
        URL = 'https://ca.finance.yahoo.com/quote/{}/balance-sheet?p={}'.format(self.ticker,self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
        bs_date_containers = html_soup.find('div', class_ = "D(tbr) C($primaryColor)")
        bs_dates = []
        
        try:
            bs_dates = bs_date_containers.find_all('span')
            
            bs_dates[0]

            x = 0
            for x in range(len(bs_dates)):
                bs_dates[x] = bs_dates[x].text
                x+=1

            try:
                recent_dates_temp.append(bs_dates[1])

            except IndexError:
                recent_dates_temp.append("Null")
                
        except AttributeError:
                recent_dates_temp.append("Null")            

            
            
        #CF stmt date
        URL = 'https://ca.finance.yahoo.com/quote/{}/cash-flow?p={}'.format(self.ticker,self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
        cf_date_containers = html_soup.find('div', class_ = "D(tbr) C($primaryColor)")
        cf_dates = []
        
        try:
            cf_dates = cf_date_containers.find_all('span')
            cf_dates[0]

            x = 0
            for x in range(len(cf_dates)):
                cf_dates[x] = cf_dates[x].text
                x+=1

            try:
                recent_dates_temp.append(cf_dates[2])
            except IndexError:
                try: 
                    recent_dates_temp.append(cf_dates[1])
                except IndexError:
                    recent_dates_temp.append("Null")
        except AttributeError:
                    recent_dates_temp.append("Null")            
                
        
        
        #IS stmt date
        URL = 'https://ca.finance.yahoo.com/quote/{}/financials?p={}'.format(self.ticker, self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')

        is_date_containers = html_soup.find('div', class_ = "D(tbr) C($primaryColor)")
        is_dates = []
        try:
            is_dates = is_date_containers.find_all('span')
            is_dates[0]

            x = 0
            for x in range(len(is_dates)):
                is_dates[x] = is_dates[x].text
                x+=1

            try:
                recent_dates_temp.append(is_dates[2])
            except IndexError:
                try: 
                    recent_dates_temp.append(is_dates[1])
                except IndexError:
                    recent_dates_temp.append("Null")
        except AttributeError:
                    recent_dates_temp.append("Null")
        
        return(recent_dates_temp)
    
    
    #define function to scrape selected balance sheet data
    #returns cash and total liabilities to a temporary variable to be extracted and later pulled into a df
    def balance_sheet_data_scrape(self):
        URL = 'https://ca.finance.yahoo.com/quote/{}/balance-sheet?p={}'.format(self.ticker,self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
 
        #find the ca$h_money
        q_totalcash_div = html_soup.find('div', text = "Total Cash")

        try:
            q_totalcash_container = q_totalcash_div.parent

            q_totalcash_values = []
            q_totalcash_values.append(q_totalcash_container.text)

            for x in q_totalcash_container.next_siblings:
                q_totalcash_values.append(x.text)

            q_totalcash_values = q_totalcash_values[1:]

            y = 0
            for x in (q_totalcash_values):
                if x == '-':
                    q_totalcash_values[y] = x.replace('-',"0")
                    q_totalcash_values[y] = int(float(q_totalcash_values[y]))

                else:
                    q_totalcash_values[y] = int(float(x.replace(',',"")))
                y+=1

        except AttributeError:
                q_totalcash_values = [0, 0, 0, 0]

        stock_data_temp.append(q_totalcash_values[0])
        
        
        #find the total liabilities
        q_totallibs_div = html_soup.find('div', text = "Total Liabilities")
    
        try: 
            q_totallibs_container = q_totallibs_div.parent

            q_totallibs_values = []
            q_totallibs_values.append(q_totallibs_container.text)

            for x in q_totallibs_container.next_siblings:
                q_totallibs_values.append(x.text)

            q_totallibs_values = q_totallibs_values[1:]

            y = 0
            for x in (q_totallibs_values):
                if x == '-':
                    q_totallibs_values[y] = x.replace('-',"0")
                    q_totallibs_values[y] = int(float(q_totallibs_values[y]))

                else:
                    q_totallibs_values[y] = int(float(x.replace(',',"")))
                y+=1

        except AttributeError:
            q_totallibs_values = [0, 0, 0, 0]
            
        stock_data_temp.append(q_totallibs_values[0])
        
        #time.sleep(30)
        return stock_data_temp
    
    
    
    #define function to pull operating cash flow
    def cashflow_data_scrape(self):
        URL = 'https://ca.finance.yahoo.com/quote/{}/cash-flow?p={}'.format(self.ticker, self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
        #op cash flow
        q_opcashflow_div = html_soup.find('div', text = "Net cash provided by operating activites")

        try:
            q_opcashflow_container = q_opcashflow_div.parent

            q_opcashflow_values = []
            q_opcashflow_values.append(q_opcashflow_container.text)

            for x in q_opcashflow_container.next_siblings:
                q_opcashflow_values.append(x.text)

            q_opcashflow_values = q_opcashflow_values[1:]

            y = 0
            for x in (q_opcashflow_values):
                if x == '-':
                    q_opcashflow_values[y] = x.replace('-',"0")
                    q_opcashflow_values[y] = int(float(q_opcashflow_values[y]))

                else:
                    q_opcashflow_values[y] = int(float(x.replace(',',"")))
                y+=1

        except AttributeError:
            q_opcashflow_values = [0, 0, 0, 0, 0]
        
        try:
            stock_data_temp.append(q_opcashflow_values[1])
        except IndexError:
            stock_data_temp.append(q_opcashflow_values[0])            
        
        #time.sleep(10)
        return stock_data_temp
    
    
    def incomestmt_data_scrape(self):
        URL = 'https://ca.finance.yahoo.com/quote/{}/financials?p={}'.format(self.ticker, self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
        #revenue
        q_rev_div = html_soup.find('div', text = "Total Revenue")

        try:
            q_rev_container = q_rev_div.parent

            q_rev_values = []
            q_rev_values.append(q_rev_container.text)

            for x in q_rev_container.next_siblings:
                q_rev_values.append(x.text)

            q_rev_values = q_rev_values[1:]

            y = 0
            for x in (q_rev_values):
                if x == '-':
                    q_rev_values[y] = x.replace('-',"0")
                    q_rev_values[y] = int(float(q_rev_values[y]))

                else:
                    q_rev_values[y] = int(float(x.replace(',',"")))
                y+=1
                
        except AttributeError:
            q_rev_values = [0, 0, 0, 0, 0]
         
        try:
            stock_data_temp.append(q_rev_values[1])
        except IndexError:
            stock_data_temp.append(q_rev_values[0])        
        
        #time.sleep(10)
        return stock_data_temp

    
    
    #define function to pull various relevant data points;
    #including Market Cap, Enterprise Value,EBITDA, D/E, BV/Share, and shares outstanding
    def otherfinancials_data_scrape(self):
        URL = 'https://ca.finance.yahoo.com/quote/{}/key-statistics?p={}'.format(self.ticker, self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        
        
        #market cap
        market_cap_div = html_soup.find(text = "Market Cap (intraday)")

        try:
            market_cap_container = market_cap_div.parent.parent

            market_cap = []
            market_cap.append(market_cap_div)

            market_cap.append(market_cap_div.parent.parent.next_sibling.text)

            formatting_tool = FormattingTool()

            if market_cap[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(market_cap[1]))
        except AttributeError:
            stock_data_temp.append(0)
            
            
        
        
        #enterprise value
        EV_div = html_soup.find(text = "Enterprise Value")

        try:
            EV_container = EV_div.parent.parent

            EV = []
            EV.append(EV_div)

            EV.append(EV_div.parent.parent.next_sibling.text)

            stock_data_temp.append(formatting_tool.value_to_float(EV[1]))
        except AttributeError:
            stock_data_temp.append(0)

        
        #EBITDA
        EBITDA_div = html_soup.find(text = "EBITDA")

        try:
            EBITDA_container = EBITDA_div.parent.parent

            EBITDA = []
            EBITDA.append(EBITDA_div)

            EBITDA.append(EBITDA_div.parent.parent.next_sibling.text)

            if EBITDA[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(EBITDA[1]))        
        except AttributeError:
            stock_data_temp.append(0)            
        
        
        #enterprise value/EBITDA
        EV_to_EBITDA_div = html_soup.find(text = "Enterprise Value/EBITDA")

        try:
            EV_to_EBITDA_container = EV_to_EBITDA_div.parent.parent

            EV_to_EBITDA = []
            EV_to_EBITDA.append(EV_to_EBITDA_div)

            EV_to_EBITDA.append(EV_to_EBITDA_div.parent.parent.next_sibling.text)


            if EV_to_EBITDA[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(EV_to_EBITDA[1]))
        
        except AttributeError:
            stock_data_temp.append(0)
        
        
        #debt to equity
        debt_to_equity_div = html_soup.find(text = "Total Debt/Equity")
        
        try:
            debt_to_equity_container = debt_to_equity_div.parent.parent

            debt_to_equity = []
            debt_to_equity.append(debt_to_equity_div)

            debt_to_equity.append(debt_to_equity_div.parent.parent.next_sibling.text)

            if debt_to_equity[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(debt_to_equity[1]))
        except AttributeError:
            stock_data_temp.append(0)
        

        #BV per share
        book_value_ps_div = html_soup.find(text = "Book Value Per Share")

        try:
            book_value_ps_container = book_value_ps_div.parent.parent

            book_value_ps = []
            book_value_ps.append(book_value_ps_div)

            book_value_ps.append(book_value_ps_div.parent.parent.next_sibling.text)

            if book_value_ps[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(book_value_ps[1]))
        except AttributeError:
            stock_data_temp.append(0)           
        
        
        #shares outstanding
        shares_outstanding_div = html_soup.find(text = "Shares Outstanding")

        try:
            shares_outstanding_container = shares_outstanding_div.parent.parent

            shares_outstanding = []
            shares_outstanding.append(shares_outstanding_div)

            shares_outstanding.append(shares_outstanding_div.parent.parent.next_sibling.text)

            if shares_outstanding[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(formatting_tool.value_to_float(shares_outstanding[1])) 
        except:
            stock_data_temp.append(0)            
        
        
        #time.sleep(15)
        return stock_data_temp

        
        
    #finally, define function to pull price at that point in time
    def price_data_scrape(self):
        URL = 'https://ca.finance.yahoo.com/quote/{}?p={}'.format(self.ticker, self.ticker)
        page = requests.get(URL)
        html_soup = BeautifulSoup(page.text, 'html.parser')
        formatting_tool = FormattingTool()
        
        try:
            stock_price_container = html_soup.find('div', class_="My(6px) Pos(r) smartphone_Mt(6px)")

            stock_price = []
            stock_price.append(dt.now())

            stock_price.append(stock_price_container.div.span.text)


            if stock_price[1] == 'N/A':
                stock_data_temp.append(0)
            else:
                stock_data_temp.append(float(stock_price[1].replace(",","")))

        except AttributeError:
            stock_data_temp.append(0)

        now = dt.now()
        date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
        stock_data_temp.append(date_time)
        
        #time.sleep(15)
        return stock_data_temp

In [ ]:
#Testing ground


stock_data = []

tsx_list_test = ['CSU.TO']

print(tsx_list_test)
    

In [ ]:
#Testing ground


stock_data_df = FormattingTool()
stock_data_df = stock_data_df.create_pd_dataframe()

y = 0
for x in tsx_list:
    
    print(x)
    #call financialdatascraper class
    analysis = FinancialDataScraper(x, stock_data)

    #initiailze df with ticker 'x'
    stock_data_df.at[y,'Ticker'] = x
    
    #scrape dates first
    stock_data_temp = analysis.financials_date_scrape()
    stock_data_df.at[y, 'BS Recent Date'] = (stock_data_temp[0])
    stock_data_df.at[y, 'CF Recent Date'] = (stock_data_temp[1])
    stock_data_df.at[y, 'IS Recent Date'] = (stock_data_temp[2])
    
    stock_data_temp = []

    
    #scrape balance sheet data    
    stock_data_temp = analysis.balance_sheet_data_scrape()
    stock_data_df.at[y,'Total Cash'] = float(stock_data_temp[0])
    stock_data_df.at[y,'Total Liabilities'] = float(stock_data_temp[1])
    
    stock_data_temp = []

   
    #scrape op cash flow data
    stock_data_temp = analysis.cashflow_data_scrape()
    stock_data_df.at[y,'Operating Cash Flow'] = float(stock_data_temp[0])
    
    stock_data_temp = []

    
    #scrape income statement data
    stock_data_temp = analysis.incomestmt_data_scrape()
    stock_data_df.at[y,'Total Revenue'] = float(stock_data_temp[0])
    
    stock_data_temp = []
    
    
    #scrape other financial data
    stock_data_temp = analysis.otherfinancials_data_scrape()
    stock_data_df.at[y,'Market Cap'] = (stock_data_temp[0])
    stock_data_df.at[y,'EV'] = stock_data_temp[1]
    stock_data_df.at[y,'EBITDA'] = stock_data_temp[2]
    stock_data_df.at[y,'EV/EBITDA'] = stock_data_temp[3]
    stock_data_df.at[y,'D/E'] = stock_data_temp[4]
    stock_data_df.at[y,'BV/Share'] = stock_data_temp[5]
    stock_data_df.at[y,'Shares Outstanding'] = stock_data_temp[6]

    stock_data_temp = []

    
    #pull price, current date/time
    stock_data_temp = analysis.price_data_scrape()
    stock_data_df.at[y,'Price'] = stock_data_temp[0]
    stock_data_df.at[y,'Date'] = stock_data_temp[1]
  
    stock_data_temp = []

    time.sleep(10)
    #loop counter
    y+=1
    

stock_data_df

In [ ]:
stock_data_df.to_csv(r'C:\Users\Mac\Desktop\Investing\Financial Data\TSX_Data_Final.csv',index=True, header=True)